# Guardrails with OpenAI Agents SDK

Guardrails validate user input before your agent processes it.

They run in parallel to your agent using a separate model. This prevents misuse of expensive agents for unrelated tasks.

The examples are taken from the official documentation: https://openai.github.io/openai-agents-python/guardrails/.

## Understanding Guardrails

Imagine you have an agent with an expensive model to help with customer requests. You don't want people to use it for unrelated tasks like math homework.

The guardrail uses a cheap and fast model to check if the input is appropriate. If the input is invalid, the guardrail stops the main agent before it runs.

## Creating a Guardrail Agent

Define the instructions for the guardrail.

The guardrail will check if queries are related to the Evidently framework.

In [ ]:
guardrail_instructions = """
Make sure the user queries are related to the Evidently framework and its documentation.

Evidently is an open-source Python library and cloud platform for evaluating, testing, and monitoring data,
AI and LLM systems. It provides evaluation metrics, testing APIs, and visual reports for model and data quality.

Examples of relevant topics:

- Create a custom LLM judge
- Customize data drift detection
- llm evaluations

Output 'fail=True' if the query is not about Evidently documentation or related topics.
Keep reasoning short (up to 10 words)
""".strip()


Define the output model for the guardrail decision.

In [ ]:
class EvidentlyDocsGuardrail(BaseModel):
    reasoning: str
    fail: bool

Create the guardrail agent with a fast model.

In [ ]:
guardrail_agent = Agent( 
    name="guardrail",
    instructions=guardrail_instructions,
    output_type=EvidentlyDocsGuardrail,
    model='gpt-4o-mini'
)


## Testing the Guardrail

Let's test the guardrail agent:

In [ ]:
result = await Runner.run(guardrail_agent, input='llm as a judge')

Output is here:

In [ ]:
result.final_output

The output shows the query is valid:

In [ ]:
EvidentlyDocsGuardrail(reasoning='Query relates to LLM evaluations in Evidently.', fail=False)

## Implementing the Input Guardrail Function

Import the required guardrail components.

In [ ]:
from agents import GuardrailFunctionOutput, input_guardrail

Define the guardrail function that wraps the guardrail agent.

This function checks if we should skip guardrail validation for continued conversations.

It returns the guardrail decision to the framework.

In [ ]:
@input_guardrail
async def guardrail(ctx, agent, messages):
    result = await Runner.run(guardrail_agent, input=messages)
    decision = result.final_output
    return GuardrailFunctionOutput(
        output_info=decision.reasoning,
        tripwire_triggered=decision.fail
    )


If we add debugging, we can see that this function is invoked when we have the max_turn exception. In these situations we don't want to trigger the guardrail:

In [ ]:
@input_guardrail
async def guardrail(ctx, agent, messages):
    if type(messages) == list and len(messages) > 1:
        return GuardrailFunctionOutput(
            output_info='no need to trigger for continued conversations',
            tripwire_triggered=False
        )
    result = await Runner.run(guardrail_agent, input=messages)
    decision = result.final_output
    return GuardrailFunctionOutput(
        output_info=decision.reasoning,
        tripwire_triggered=decision.fail
    )


## Adding Guardrails to the Search Agent

Update the search agent to include the input guardrail.

In [ ]:
agent_tools = [
    function_tool(tools.search),
    function_tool(tools.read_file)
]

search_agent = Agent(
    name='search',
    tools=agent_tools,
    instructions=search_instructions,
    input_guardrails=[guardrail],
    model=config.model,
    output_type=SearchResultArticle,
)


## Handling Guardrail Exceptions

Import the guardrail exception.

In [ ]:
from agents.exceptions import InputGuardrailTripwireTriggered

Update the run_stream function to handle guardrail failures.

In [ ]:
async def run_stream(agent, input, handler, max_turns=3):
    try:
        result = Runner.run_streamed(
            agent,
            input=input,
            max_turns=max_turns
        )
        
        parser = StreamingJSONParser(handler)

        async for event in result.stream_events():
            if event.type == "run_item_stream_event":
                if event.item.type == "tool_call_item":
                    tool_call = event.item.raw_item
                    f_name = tool_call.name
                    args = tool_call.arguments
                    print(f"TOOL CALL ({event.item.agent.name}): {f_name}({args})")
            
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                parser.parse_incremental(event.data.delta)

        return result
    except MaxTurnsExceeded as e:
        print('too many turns')
        finish_prompt = 'System message: The number of searches has exceeded the limit. Proceed to finishing the writeup'
        finish_message = [{'role': 'user', 'content': finish_prompt}]
        messages = result.to_input_list() + finish_message
        final_result = await run_stream(agent, input=messages, handler=handler, max_turns=1)
        return final_result
    except InputGuardrailTripwireTriggered as e:
        run_data = e.run_data
        for input_guardrail in run_data.input_guardrail_results:
            o = input_guardrail.output
            if o.tripwire_triggered:
                print(o.output_info)
        return e.run_data


## Testing the Complete System

Test the agent with an invalid query.

The guardrail should block the query and display the reasoning.



In [ ]:
result = await run_stream(search_agent, 'how much is sqrt(pi)', SearchResultHandler())

Output shows the guardrail blocked the request:

```text
TOOL CALL (search): search({"query":"sqrt(pi) value"})
Not related to Evidently framework.
```

Done! Now we guardrail our application againts unwanted queries.